## 42.1 데코레이터 만들기 
- 데코레이터란? : 함수(메서드)를 장식하는 도구
- ex) @staticmethod, @classmetod, @abstractmethod 등 
- 함수를 수정하지 않은 상태에서 추가 기능을 구현할 때 사용함  

**일반 함수**

In [1]:
def hello():
    print('hello 함수 시작')
    print('hello')
    print('hello 함수 끝')
    
def world():
    print('world 함수 시작')
    print('world')    
    print('world 함수 끝') 

hello()
world()

hello 함수 시작
hello
hello 함수 끝
world 함수 시작
world
world 함수 끝


In [3]:
def trace(func):   # 함수를 매개변수로 받음
    def wrapper(): # 호출할 함수를 감싸는 함수 -> 왜 감싸지? : 해당 함수의 시작과 끝을 출력할 수 있기 때문에 
        print(func.__name__, '함수 시작')   #.__name__ : 함수 이름 출력 
        func()
        print(func.__name__, '함수 끝')
    return wrapper

def hello():
    print('hello')
    
def world():
    print('world')    

trace_hello = trace(hello)
trace_hello()
trace_world = trace(world)
trace_world()

hello 함수 시작
hello
hello 함수 끝
world 함수 시작
world
world 함수 끝


In [4]:
def trace(func):   # 함수를 매개변수로 받음
    def wrapper(): # 호출할 함수를 감싸는 함수 -> 왜 감싸지? : 해당 함수의 시작과 끝을 출력할 수 있기 때문에 
        print(func.__name__, '함수 시작')   #.__name__ : 함수 이름 출력 
        func()
        print(func.__name__, '함수 끝')
    return wrapper
@trace
def hello():
    print('hello')
@trace    
def world():
    print('world')    

hello()
world()

hello 함수 시작
hello
hello 함수 끝
world 함수 시작
world
world 함수 끝


- 함수를 감싸는 형태로 구성
- 기존 함수를 수정하지 않으면서 추가 기능을 구현할 때 사용
## 42.2 매개변수와 반환값을 처리하는 데코레이터 만들기 

In [7]:
def trace(func):   # 함수를 매개변수로 받음
    def wrapper(a, b): # 호출할 함수의 매개변수와 똑같이 지정
        r = func(a, b)
        print('{0}(a={1}, b={2}) -> {3}'.format(func.__name__, a, b, r))  # 매개변수와 반환값 출력
        return r # add함수의 return값을 반환하지 않으면, add함수를 호출해도 return값을 반환받지 못한다. 
    return wrapper

@trace
def add(a, b):
    return a+b

print(add(10, 20))

add(a=10, b=20) -> 30
30


**가변 인수 함수 데코레이터**
- 매개변수의 갯수가 정해지지 않은 함수의 데코레이터
- *args, ** kwargs
    - 위치 인수, 키워드 인수
    - list, dictionary

In [10]:
def trace(func):   # 함수를 매개변수로 받음
    def wrapper(*args, **kwargs): # 위치 인수와 키워드 인수를 모두 받을 수 있도록 한다. 
        r = func(*args, **kwargs)  # unpacking
        print('{0}(args={1}, kwargs={2}) -> {3}'.format(func.__name__, args, kwargs, r))  # 매개변수와 반환값 출력
        return r # add함수의 return값을 반환하지 않으면, add함수를 호출해도 return값을 반환받지 못한다. 
    return wrapper

@trace
def get_max(*args): # 위치 인수를 사용
    return max(args)

@trace
def get_min(**kwargs):  # 키워드 인수를 사용
    return min(kwargs.values())

print(get_max(10, 20))
print(get_min(x=10, y=20, z=30))

get_max(args=(10, 20), kwargs={}) -> 20
20
get_min(args=(), kwargs={'x': 10, 'y': 20, 'z': 30}) -> 10
10


## 42.3 매개변수가 있는 데코레이터 만들기
- 값을 지정해서 동작을 바꿀 수 있다.

In [1]:
def is_multiple(x):  # 데코레이터가 사용할 매개변수를 지정 
    def real_decorator(func):
        def wrapper(a,b):
            r = func(a,b)
            if r % x == 0 :
                print('{0}의 반환값은 {1}의 배수입니다.'.format(func.__name__, x))
            else:
                print('{0}의 반환값은 {1}의 배수가 아닙니다.'.format(func.__name__, x))
            return r
        return wrapper
    return real_decorator

@is_multiple(3)  # @데코레이터(인수)
def add(a, b):
    return a+b

print(add(10, 20))
print(add(2, 5))

add의 반환값은 3의 배수입니다.
30
add의 반환값은 3의 배수가 아닙니다.
7


## 42.4 클래스로 데코레이터 만들기
- 인스턴스를 함수처럼 호출하게 해주는 __ call __ 메서드를 구현해야 함
- 클로저 형태의 데코레이터와 같이 @데코레이터 형식으로 사용한다. 

In [7]:
class Trace:
    def __init__(self, func): # 호출할 함수를 인스턴스의 초깃값으로 받음
        self.func = func   # 호출할 함수를 속성 func에 저장
        
    def __call__(self):   
        print(self.func.__name__, '함수 시작')   #.__name__ : 함수 이름 출력 
        self.func()  # 함수(func)호출
        print(self.func.__name__, '함수 끝')

@Trace
def christmas():
    print('moon')
    
christmas()

christmas 함수 시작
moon
christmas 함수 끝


## 42.5 클래스로 매개변수와 반환값을 처리하는 데코레이터 만들기 

In [9]:
class Trace:
    def __init__(self, func):   
        self.func = func
        
    def __call__(self, *args, **kwargs): # 위치 인수와 키워드 인수를 모두 받을 수 있도록 한다. 
        r = self.func(*args, **kwargs)  # unpacking
        print('{0}(args={1}, kwargs={2}) -> {3}'.format(self.func.__name__, args, kwargs, r))  # 매개변수와 반환값 출력
        
        return r # add함수의 return값을 반환하지 않으면, add함수를 호출해도 return값을 반환받지 못한다. 

@Trace
def add(a, b): 
    return a+b

print(add(10, 20))
print(add(b=29, a=-5))

add(args=(10, 20), kwargs={}) -> 30
30
add(args=(), kwargs={'b': 29, 'a': -5}) -> 24
24


- 클래스로 매개변수가 있는 데코레이터 만들기

In [14]:
class Is_multiple:
    def __init__(self, x):
        self.x = x
        
    def __call__(self, func):
        def wrapper(a, b):
            r = func(a, b)
            if r % self.x == 0:
                print('{0}의 반환값은 {1}의 배수입니다.'.format(func.__name__, self.x))
            else:
                print('{0}의 반환값은 {1}의 배수가 아닙니다.'.format(func.__name__, self.x))
            return r
        return wrapper

@Is_multiple(3)  # @데코레이터(인수)
def add(a, b):
    return a+b

print(add(10, 20))
print(add(2, 5))

add의 반환값은 3의 배수입니다.
30
add의 반환값은 3의 배수가 아닙니다.
7


### 데코레이터의 용도
- 디버깅
- 함수 성능 측정
- 함수 실행 전 데이터 확인 

## 42.7 연습문제: 데코레이터로 매개변수의 자료형 검사하기
다음 소스 코드에서 데코레이터 type_check를 작성하세요. type_check는 함수의 매개변수가 지정된 자료형(클래스)이면 함수를 정상적으로 호출하고, 지정된 자료형과 다르면 RuntimeError 예외를 발생시키면서 '자료형이 다릅니다.' 에러 메시지를 출력해야 합니다. 여기서 type_check에 지정된 첫 번째 int는 호출할 함수에서 첫 번째 매개변수의 자료형을 뜻하고, 두 번째 int는 호출할 함수에서 두 번째 매개변수의 자료형을 뜻합니다.


__class사용__

In [24]:
class type_check:
    def __init__(self, type_a, type_b):
        self.type_a = type_a
        self.type_b = type_b
        
    def __call__(self, func):
        def wrapper(a, b):
            r = func(a, b)
            if type(a)==self.type_a and type(b)==self.type_b:
                return r
            else:
                raise RuntimeError('자료형이 다릅니다.')
        return wrapper
        
                
        

@type_check(int, int)
def add(a, b):
    return a + b
 
print(add(10, 20))
print(add('hello', 'world'))

30


RuntimeError: 자료형이 다릅니다.

__type(a)로 검사__

In [25]:
def type_check(type_a, type_b):  # 데코레이터가 사용할 매개변수를 지정 
    def real_decorator(func):
        def wrapper(a,b):
            r = func(a,b)
            if type(a)==type_a and type(b)==type_b :
                return r
            else:
                raise RuntimeError('자료형이 다릅니다.')            
        return wrapper
    return real_decorator

@type_check(int, int)
def add(a, b):
    return a + b
 
print(add(10, 20))
print(add('hello', 'world'))

30


RuntimeError: 자료형이 다릅니다.

__정답__
- isinstance() 사용
- func(a, b)를 변수에 저장하지 않고 바로 반환

In [26]:
def type_check(type_a, type_b):  # 데코레이터가 사용할 매개변수를 지정 
    def real_decorator(func):
        def wrapper(a,b):
            if isinstance(a, type_a) and isinstance(b, type_b) :
                return func(a, b)
            else:
                raise RuntimeError('자료형이 다릅니다.')            
        return wrapper
    return real_decorator

@type_check(int, int)
def add(a, b):
    return a + b
 
print(add(10, 20))
print(add('hello', 'world'))

30


RuntimeError: 자료형이 다릅니다.

## 42.8 심사문제: HTML 태그 데코레이터 만들기
표준 입력으로 HTML 태그 이름 두 개가 입력됩니다. 다음 소스 코드에서 함수의 반환값을 HTML 태그로 감싸는 데코레이터를 만드세요. HTML 태그는 웹 페이지에 사용하는 문법이며 \<span>문자열\</span>, \<p>문자열\</p>처럼 <태그이름>으로 시작하며 </태그이름>으로 끝납니다.

In [36]:
class html_tag:
    def __init__(self, tag):
        self.tag = tag
        
    def __call__(self, func):
        def wrapper():
            r = func()
            print('<{0}>{1}</{0}>'.format(self.tag, r))  # 값 저장이 안된다. 
            return '<{0}>{1}</{0}>'.format(self.tag, r)
        
        return wrapper
        
                

a, b = input().split()
 
@html_tag(a)
@html_tag(b)
def hello():
    return 'Hello, world!'
 
print(hello())

b i
<i>Hello, world!</i>
<b><i>Hello, world!</i></b>
<b><i>Hello, world!</i></b>


- 가장 가까운 데코레이터부터 실행  
- return 으로 값을 저장후 다시 실행

In [33]:
def html_tag(tag):  # 데코레이터가 사용할 매개변수를 지정 
    def real_decorator(func):
        def wrapper():
            r = func()
            return '<{0}>{1}</{0}>'.format(tag, r)
                     
        return wrapper
    return real_decorator

a, b = input().split()
 
@html_tag(a)
@html_tag(b)
def hello():
    return 'Hello, world!'
 
print(hello())

b i
<b><i>Hello, world!</i></b>
